In [1]:
import pandas as pd

In [2]:
df_readme = pd.read_csv('../java_nonpersonal_readme.csv')

In [4]:
df_readme = df_readme[(df_readme.stars != 0) & (df_readme.forked_from == 0)].reset_index(drop=True)

In [5]:
df_readme

,id,name,language,url,forked_from,owner_id,stars,readme
0,33,Glomes,Java,https://api.github.com/repos/SnowblindFatal/Gl...,0,140,4,Glomes is (was?) a project made by J. Saarela ...
1,147,ActionBarSherlock,Java,https://api.github.com/repos/JakeWharton/Actio...,0,896,8330,# DEPRECATED\n\nActionBarSherlock is deprecate...
2,190,SimianArmy,Java,https://api.github.com/repos/Netflix/SimianArmy,0,1600,6714,[![NetflixOSS Lifecycle](https://img.shields.i...
3,252,rstudio,Java,https://api.github.com/repos/rstudio/rstudio,0,1904,2731,RStudio\n=====================================...
4,313,IGV,Java,https://api.github.com/repos/igvteam/igv,0,2076,133,# igv\n[![Build Status](https://travis-ci.org/...
...,...,...,...,...,...,...,...,...
142466,137562943,Notes,Java,https://api.github.com/repos/DaveAAA/Notes,0,41194524,2,# Notes.\nNotes is a mobile application for ta...
142467,137574913,SocketChat,Java,https://api.github.com/repos/DemS98/SocketChat,0,46417714,1,# SocketChat\nSocketChat consists in two appli...
142468,137584454,SebUzduotis,Java,https://api.github.com/repos/MartinCepulionis/...,0,51559454,1,# SebUzduotis\n\nAfter you input two integers(...
142469,137596645,adoptopenjdk-plugin,Java,https://api.github.com/repos/jenkinsci/adoptop...,0,2206,1,Jenkins AdoptOpenJDK installer Plugin\n=======...


In [10]:
df_sample_metrics_calculated = pd.read_csv('../metrics_nonezero_nonefork_v7.csv', index_col=0)

In [24]:
from bs4 import BeautifulSoup
# from markdown import markdown
# from markdown2 import markdown
from mistune import markdown
import spacy

import json

import time

from functools import reduce

import csv

import math


import nltk

from nltk.stem.porter import PorterStemmer

import re
import string
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()

In [21]:
SPACY_NLP = spacy.load("en_core_web_sm")
stemmer = PorterStemmer()

In [22]:
def list_contain_kw_direct(lst, kw):
    for item in lst:
        if kw in item:
            return 1
    return 0


def list_kw_index(lst, kw):
    for i, item in enumerate(lst):
        for token in item.split():
            clean_token = token.replace('-', '').replace('?', '').replace('.', '').replace(':', '')
            if stemmer.stem(clean_token).startswith(stemmer.stem(kw)):
                #         print(token, kw)
                return i

    #   kw = ''.join(remove_special_chars(kw))
    #   if not kw:
    #     return -1

    #   for i, item in enumerate(lst):
    #     resonstructed_name = item.replace('-','').replace('_','').replace(' ','').replace('.','').replace(':','')
    #     if kw in resonstructed_name or resonstructed_name in kw:
    #       if not resonstructed_name:
    #         continue
    #       return i

    return -1


def header_contain_project_name(lst, kw):
    kw = ''.join(remove_special_chars(kw))

    if not kw:
        return 0

    for item in lst:
        resonstructed_name = item.replace('-', '').replace('_', '').replace(' ', '').replace('.', '').replace(':', '')
        #     print(resonstructed_name)

        if not resonstructed_name:
            continue

        if kw in resonstructed_name or resonstructed_name in kw:
            return 1
    return 0


def list_contains_exact(lst, kw):
    for item in lst:
        for token in item.split():
            if token == kw:
                return 1

    return 0


def list_contains(lst, kw):
    for item in lst:
        for token in item.split():
            clean_token = token.replace('-', '').replace('?', '').replace('.', '').replace(':', '')
            if stemmer.stem(clean_token).startswith(stemmer.stem(kw)):
                #       if token.startswith(kw):
                return 1

    return 0


def preprocess_readme(readme_md):
    html = md2html(readme_md)

    rm_clean = convert_markdown(html)

    nocode = rm_clean

    for code in find_all_codes(readme_md):
        rm_clean = rm_clean.replace(code[1], '')

    return nocode, rm_clean


def find_all_badges(md):
    t_list = BeautifulSoup(markdown(md)).find_all('img')

    # print(t_list)

    rv = []

    for img in t_list:
        c = str(img).lower()
        if 'badge' in c:
            rv.append(img)
        elif 'svg' in c:
            if 'hack23' in c or 'travis' in c or 'coveralls' in c or 'shields.io' in c or 'davideas':
                rv.append(img)

    return rv


def find_all_urls(md):
    t_list = BeautifulSoup(markdown(md)).find_all('a')
    return t_list


def find_all_images(md):
    t_list = BeautifulSoup(markdown(md)).find_all('img')
    return t_list


def find_all_blocks(md):
    html = BeautifulSoup(markdown(md))
    t_list = html.find_all(re.compile('^h[1-6]$'))
    headers = [(x.name, x.text.lower()) for x in t_list]

    blocks = []

    i = 0
    moving = ''

    for child in html.recursiveChildGenerator():
        if child.name:
            #       print(child.name, child.text)
            #       if child.text:
            if i == len(headers):
                moving += str(child)
            elif child.name == headers[i][0] and child.text.lower() == headers[i][1]:
                blocks.append(moving)
                i += 1
                moving = ''
            else:
                moving += str(child) + '\n'
    blocks.append(moving)

    return blocks


def find_all_codes(md):
    t_list = BeautifulSoup(markdown(md)).find_all('code')
    return [(x.name, x.text) for x in t_list]


def find_all_headers(md):
    t_list = BeautifulSoup(markdown(md)).find_all(re.compile('^h[1-6]$'))
    return [(x.name, x.text.lower()) for x in t_list]


def remove_special_chars(readme):
    return list(map(lambda x: re.sub("[^a-zA-Z ]", "", x), readme))


def filter_readme(readme, lemmatize=True, lowercase=True, remove_stopwords=True,
                  remove_punctuation=True, remove_digits=True, remove_space=True,
                  remove_like_num=True, remove_short=True,
                  remove_contains_num=True, remove_nonenglish=True,
                  remove_hyperlink=True):
    doc = SPACY_NLP(readme)  # Create Spacy Document
    isEnglishReadme = True
    filtered_tokens = []
    for token in doc:
        # Exclude tokens from stopwords list
        if remove_stopwords and token.is_stop:
            continue
        # Exclude tokens from punctuations list
        elif remove_punctuation and token.is_punct:
            continue
        # Exclude tokens that are digits
        elif remove_digits and token.is_digit:
            continue
        # Exclude whitespace characters
        elif remove_space and token.is_space:
            continue
        # Exclude tokens that are similar to numbers (e.g., "ten")
        elif remove_like_num and token.like_num:
            continue
        # Exclude tokens that are shorter than 3 characters
        elif remove_short and len(token) < 3:
            continue
        # Exclude tokens that contain a number
        elif remove_contains_num and not re.search("[0-9]", token.text) is None:
            continue
        elif remove_nonenglish and not isEnglish(token.text):
            isEnglishReadme = False
            continue
        elif remove_hyperlink and not re.search(r'((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)',
                                                token.text) is None:
            continue
        else:
            if lemmatize:  # Return lemma
                filtered_token = token.lemma_
            else:
                filtered_token = token.text

            if lowercase:  # Return lowercased token
                filtered_tokens += [filtered_token.lower()]
            else:
                filtered_tokens += [filtered_token]

    return filtered_tokens, isEnglishReadme


def convert_markdown(text):
    rv = ''.join(text.findAll(text=True))
    return rv


def md2html(md):
    return BeautifulSoup(markdown(md))


def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        #         print(s)
        return False
    else:
        return True


SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]


def get_readme_metrics(project_name, readme_md):
    if isinstance(project_name, float) and math.isnan(project_name):
        project_name = ""

    headers_pair = find_all_headers(readme_md)

    headers = [t for n, t in headers_pair]

    codes_pair = find_all_codes(readme_md)
    codes = [t for n, t in codes_pair]

    blocks = find_all_blocks(readme_md)

    images = find_all_images(readme_md)

    urls = find_all_urls(readme_md)

    filtered_readme = list(map(lambda x: filter_readme(convert_markdown(BeautifulSoup(x))), blocks))
    # first tokenlized readme blocks and booleans indicating if each block is pure English
    tokenlized_blocks, isEnglish = list(zip(*filtered_readme))

    isEnglish = list(isEnglish)
    tokenlized_blocks = list(tokenlized_blocks)

    blocks_token_counts = list(map(len, tokenlized_blocks))[-len(headers):]
    blocks_token_counts_np = np.array(blocks_token_counts)

    if blocks_token_counts_np.size == 0:
        blocks_token_counts_np = np.array([0])

    blocks_token_counts_min = np.min(blocks_token_counts_np)
    blocks_token_counts_max = np.max(blocks_token_counts_np)
    blocks_token_counts_mean = np.mean(blocks_token_counts_np)
    blocks_token_counts_median = np.median(blocks_token_counts_np)

    #   clean_readme, original_readme = preprocess_readme(readme_md)

    #   clean_readme = clean_readme.lower()

    #   readme_tokens = filter_readme(clean_readme)

    #   readme_tokens = remove_special_chars(readme_tokens)

    aligned_headers = headers.copy()
    aligned_headers.insert(0, "")

    # merge blocked with the headers for checking if entire readme is English only
    isEnglish = isEnglish + [filter_readme(h)[1] for h in aligned_headers]
    isEnglish = 1 if all(isEnglish) else 0

    reconstructed_readme_tokens = [filter_readme(h)[0] + c for h, c in zip(aligned_headers, tokenlized_blocks)]

    reconstructed_readme_tokens = reduce(lambda x, y: x + y, reconstructed_readme_tokens)

    readme_tokens = reconstructed_readme_tokens

    readme_body_tokens = reduce(lambda x, y: x + y, tokenlized_blocks)

    # header only keyword search
    isNameInReadmeHeaders = list_contains(headers,
                                          project_name.lower())  # or header_contain_project_name(headers, project_name.lower())

    isNameHeaderNotEmpty = 0
    if isNameInReadmeHeaders:
        name_idx = list_kw_index(aligned_headers, project_name.lower())
        #     print(name_idx)

        if name_idx == -1:
            raise Exception('Unable to find the index of project name header!')

        #     print(tokenlized_blocks)
        isNameHeaderNotEmpty = 0 if len(tokenlized_blocks[name_idx]) == 0 else 1

    isDescriptionKWInReadmeHeaders = list_contains(headers, 'describe') or list_contains(headers,
                                                                                         'description') or list_contains(
        headers, 'summary') or list_contains(headers, 'overview') or list_contains(headers,
                                                                                   'introduction') or list_contains(
        headers, 'about') or list_contains(headers, 'introduce') or list_contain_kw_direct(headers, 'what is')
    isContentKWInReadmeHeaders = list_contains(headers, 'content')
    isInstallationKWInReadmeHeaders = list_contains(headers, 'install') or list_contains(headers,
                                                                                         'build') or list_contains(
        headers, 'setup') or list_contains(headers, 'download') or list_contains(headers, 'compile')
    #   isUsageKWInReadmeHeaders = list_contains(headers, 'use') or list_contains(headers, 'usage') or list_contains(headers, 'customize') or list_contains(headers, 'quickstart') or list_contains(headers, 'run') or list_contains(headers, 'start') or list_contain_kw_direct(headers, 'getting started') or list_contain_kw_direct(headers, 'quick guide')
    isUsageKWInReadmeHeaders = list_contains(headers, 'use') or list_contains(headers, 'usage') or list_contains(
        headers, 'quickstart') or list_contains(headers, 'run') or list_contains(headers, 'start')

    isContributingKWInReadmeHeaders = list_contains(headers, 'contribute')
    isCreditsKWInReadmeHeaders = list_contains(headers, 'credit') or list_contains(headers, 'acknowledge')
    isLicenseKWInReadmeHeaders = list_contains(headers, 'license') or list_contains(headers,
                                                                                    'licence') or list_contains(headers,
                                                                                                                'copyright')
    isTestKWInReadmeHeaders = list_contains(headers, 'test')

    isDocumentKWInReadmeHeaders = list_contains(headers, 'document') or list_contains(headers, 'docs')

    isArchiveKWInReadmeHeaders = list_contains(headers, 'archive')

    isExampleKWInReadmeHeaders = list_contains(headers, 'example') or list_contains(headers, 'demo') or list_contains(
        headers, 'sample')

    isAuthorKWInReadmeHeaders = list_contains_exact(headers, 'author') or list_contains_exact(headers,
                                                                                              'authors') or list_contain_kw_direct(
        headers, 'developed by') or list_contain_kw_direct(headers, 'created by')

    isTroubleshootKWInReadmeHeaders = list_contains(headers, 'troubleshoot')

    isScreenshotKWInReadmeHeaders = list_contains(headers, 'screenshot')

    isDeprecateKWInReadmeHeaders = list_contains(headers, 'deprecate') or list_contains(headers, 'retire')

    isMovedKWInReadmeHeaders = list_contain_kw_direct(headers, 'moved')

    isFeatureKWInReadmeHeaders = list_contains(headers, 'feature')

    isConfigureKWInReadmeHeaders = list_contains(headers, 'configure')

    isDependencyKWInReadmeHeaders = list_contains(headers, 'dependency') or list_contains(headers, 'requirement')

    isReferenceKWInReadmeHeaders = list_contains(headers, 'reference')

    isReleaseKWInReadmeHeaders = list_contains(headers, 'release')

    isChangelogInReadmeHeaders = list_contains(headers, 'changelog')

    isQuestionInReadmeHeaders = list_contains(headers, 'question')

    isIssueInReadmeHeaders = list_contains(headers, 'issue')

    isNoteInReadmeHeaders = list_contains(headers, 'note') or list_contains(headers, 'remark')

    # body only keyword search
    isDeprecateKWInReadmeBody = list_contains(readme_body_tokens, 'deprecate') or list_contains(readme_body_tokens,
                                                                                                'retire')

    isMovedKWInReadmeBody = list_contain_kw_direct(readme_body_tokens, 'moved')

    isLicenseKWInReadmeBody = list_contains(readme_body_tokens, 'license') or list_contains(readme_body_tokens,
                                                                                            'licence') or list_contains(
        readme_body_tokens, 'copyright')

    isAuthorKWInReadmeBody = list_contains_exact(readme_body_tokens, 'author') or list_contains_exact(
        readme_body_tokens, 'authors') or list_contain_kw_direct(readme_body_tokens,
                                                                 'developed by') or list_contain_kw_direct(
        readme_body_tokens, 'created by')

    metrics = {
        #       'reconstructed_readme_tokens': len(reconstructed_readme_tokens),
        'isEnglishReadme': isEnglish,
        'readme_header_count': len(headers),
        'readme_code_block_count': len(codes),
        'readme_image_count': len(images),
        'readme_url_count': len(urls),
        #       'blocks_token_counts_min': blocks_token_counts_min,
        #       'blocks_token_counts_max': blocks_token_counts_max,
        #       'blocks_token_counts_mean': blocks_token_counts_mean,
        #       'blocks_token_counts_median': blocks_token_counts_median,
        'project_name_header_kw': isNameInReadmeHeaders,
        'project_name_header_non_empty': isNameHeaderNotEmpty,
        'description_header_kw': isDescriptionKWInReadmeHeaders,
        'content_header_kw': isContentKWInReadmeHeaders,
        'install_header_kw': isInstallationKWInReadmeHeaders,
        'usage_header_kw': isUsageKWInReadmeHeaders,
        'contribut_header_kw': isContributingKWInReadmeHeaders,
        'credit_header_kw': isCreditsKWInReadmeHeaders,
        'license_header_kw': isLicenseKWInReadmeHeaders,
        'test_header_kw': isTestKWInReadmeHeaders,
        'document_header_kw': isDocumentKWInReadmeHeaders,
        'archive_header_kw': isArchiveKWInReadmeHeaders,
        'example_header_kw': isExampleKWInReadmeHeaders,
        'author_header_kw': isAuthorKWInReadmeHeaders,
        'troubleshoot_header_kw': isTroubleshootKWInReadmeHeaders,
        'screenshot_header_kw': isScreenshotKWInReadmeHeaders,
        'deprecate_header_kw': isDeprecateKWInReadmeHeaders,
        'moved_header_kw': isMovedKWInReadmeHeaders,
        'feature_header_kw': isFeatureKWInReadmeHeaders,
        'configure_header_kw': isConfigureKWInReadmeHeaders,
        'dependency_header_kw': isDependencyKWInReadmeHeaders,
        'reference_header_kw': isReferenceKWInReadmeHeaders,
        'release_header_kw': isReleaseKWInReadmeHeaders,
        'changelog_header_kw': isChangelogInReadmeHeaders,
        'question_header_kw': isQuestionInReadmeHeaders,
        'issue_header_kw': isIssueInReadmeHeaders,
        'note_header_kw': isNoteInReadmeHeaders,

        'license_body_kw': isLicenseKWInReadmeBody,
        'author_body_kw': isAuthorKWInReadmeBody,

        'deprecate_body_kw': isDeprecateKWInReadmeBody,
        'moved_body_kw': isMovedKWInReadmeBody,

        #       'readme_tokens':readme_tokens,
        'readme_tokens_count': len(readme_tokens)
    }

    #   return pd.DataFrame(metrics, index=['thisisaindex']).transpose().thisisaindex
    return pd.Series(list(metrics.values()), index=list(metrics.keys()))


In [ ]:
df_sample_metrics = df_readme.progress_apply(lambda x: get_readme_metrics(x['name'], x.readme), axis=1)